In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [2]:
data = pd.read_csv('../clean_data2.csv')
data.head(20)

,Classification,Market,Wholesale,Retail,Supply Volume,County,Date
0,Mixed-Traditional,Eldama Ravine,31.288,40.00,2650.0,Baringo,2021-05-24
1,Mixed-Traditional,Eldama Ravine,27.780,40.00,1000.0,Baringo,2021-05-24
2,Mixed-Traditional,Eldama Ravine,22.220,40.00,900.0,Baringo,2021-05-24
3,Mixed-Traditional,Eldama Ravine,31.110,45.00,900.0,Baringo,2021-06-28
4,Mixed-Traditional,Eldama Ravine,31.110,32.00,900.0,Baringo,2021-09-20
5,Mixed-Traditional,Eldama Ravine,30.000,35.86,900.0,Baringo,2021-10-04
6,Mixed-Traditional,Eldama Ravine,30.000,36.00,900.0,Baringo,2021-10-25
7,Mixed-Traditional,Eldama Ravine,30.000,36.00,600.0,Baringo,2022-01-03
8,Mixed-Traditional,Eldama Ravine,30.000,34.00,1800.0,Baringo,2022-01-31
9,Mixed-Traditional,Eldama Ravine,30.000,34.00,1500.0,Baringo,2022-02-07


In [3]:
# Assume 'data' is the main dataset with 'Date', 'Wholesale', and 'County'
data = data[['Date', 'Wholesale', 'County']]
data['Wholesale'] = np.log(data['Wholesale'])  # Log transformation for variance stabilization

# Filter for 'Baringo' county and aggregate by date
baringo = data[data['County'] == 'Baringo']
baringo = baringo.groupby('Date', as_index=False).agg({'Wholesale': 'mean'})

# Step 1: Convert 'Date' to datetime format and set as index
baringo['Date'] = pd.to_datetime(baringo['Date'])
baringo.set_index('Date', inplace=True)

# Step 2: Ensure daily frequency and interpolate missing dates
daily_data = baringo.asfreq('D').interpolate(method='linear')

# Step 3: Differencing and ADF test for stationarity
baringo_diff = daily_data.diff().dropna()
result = adfuller(baringo_diff['Wholesale'])
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")
print(f"Critical Value for 5% significance: {result[4]['5%']}")

# Step 4: Train-test split
train_size = int(len(baringo_diff) * 0.8)
train_data, test_data = baringo_diff[:train_size], baringo_diff[train_size:]

# Step 5: Fit the ARIMA model on the differenced data
model = ARIMA(train_data['Wholesale'], order=(1, 1, 0))
arima_model = model.fit()
print(arima_model.summary())

# Set the forecast length to match the test data length
forecast_steps = len(test_data)

# Make predictions on the differenced data
forecast_diff = arima_model.forecast(steps=forecast_steps)

# Convert the forecast back to the log-transformed scale by adding the last train value
last_train_value = daily_data['Wholesale'].iloc[train_size - 1]
forecast_log = forecast_diff.cumsum() + last_train_value

# Revert log transformation to get back to the original scale
forecast_original = np.exp(forecast_log)

# Ensure that test_data_original has the exact same length as forecast_original
test_data_original = np.exp(daily_data['Wholesale'].iloc[train_size:train_size + forecast_steps])

# Calculate evaluation metrics
mae = mean_absolute_error(test_data_original, forecast_original)
mse = mean_squared_error(test_data_original, forecast_original)
rmse = np.sqrt(mse)
mape = (abs((test_data_original - forecast_original) / test_data_original).mean()) * 100

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


ADF Statistic: -10.309467212586949
p-value: 3.213630743966811e-18
Critical Value for 5% significance: -2.8650159190118694
                               SARIMAX Results                                
Dep. Variable:              Wholesale   No. Observations:                  677
Model:                 ARIMA(1, 1, 0)   Log Likelihood                 329.776
Date:                Thu, 14 Nov 2024   AIC                           -655.553
Time:                        20:47:44   BIC                           -646.521
Sample:                    05-25-2021   HQIC                          -652.056
                         - 04-01-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0031      0.104     -0.030      0.976      -0.206       0.200
sigma2   

In [5]:
# Check for constant values or NaNs in the Wholesale column
if daily_data['Wholesale'].isnull().any():
    print("Missing values detected. Please handle them before running the ADF test.")
    
if daily_data['Wholesale'].nunique() == 1:
    print("Constant values detected. ADF test may not work on constant data.")


In [6]:
import os
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Make sure the folder for saving models exists
os.makedirs("county_timeseries_models", exist_ok=True)

# Log-transform the 'Wholesale' column for variance stabilization
data['Wholesale'] = np.log(data['Wholesale'])

# Get list of unique counties
counties = data['County'].unique()

# Loop over each county
for county in counties:
    # Filter for the specific county and aggregate by date
    county_data = data[data['County'] == county]
    county_data = county_data.groupby('Date', as_index=False).agg({'Wholesale': 'mean'})

    # Step 1: Convert 'Date' to datetime format and set as index
    county_data['Date'] = pd.to_datetime(county_data['Date'])
    county_data.set_index('Date', inplace=True)

    # Step 2: Ensure daily frequency and interpolate missing dates
    # daily_data = county_data.asfreq('D').interpolate(method='linear')
    # Example of using polynomial interpolation
    daily_data = county_data.asfreq('D').interpolate(method='polynomial', order=2)


    # Step 3: Check for stationarity and apply differencing if necessary
    adf_result = adfuller(daily_data['Wholesale'], maxlag=3)
    diff_required = adf_result[1] > 0.05  # If p-value > 0.05, data is non-stationary
    
    if diff_required:
        data_to_model = daily_data.diff().dropna()
    else:
        data_to_model = daily_data

    # Step 4: Train-test split
    train_size = int(len(data_to_model) * 0.8)
    train_data, test_data = data_to_model[:train_size], data_to_model[train_size:]
    forecast_steps = len(test_data)

    # Step 5: Fit the ARIMA model on the prepared data
    model = ARIMA(train_data['Wholesale'], order=(1, 1, 0)) if diff_required else ARIMA(train_data['Wholesale'], order=(1, 0, 0))
    arima_model = model.fit()

    # Save model summary to a text file
    with open(f"county_timeseries_models/{county}_model_summary.txt", "w") as f:
        f.write(arima_model.summary().as_text())

    # Make predictions on the differenced data
    forecast_diff = arima_model.forecast(steps=forecast_steps)

    # Convert forecast back to the log-transformed or original scale
    if diff_required:
        last_train_value = daily_data['Wholesale'].iloc[train_size - 1]
        forecast_log = forecast_diff.cumsum() + last_train_value
    else:
        forecast_log = forecast_diff

    forecast_original = np.exp(forecast_log)

    # Prepare test data in the original scale for evaluation
    test_data_original = np.exp(daily_data['Wholesale'].iloc[train_size:train_size + forecast_steps])

    # Step 6: Calculate evaluation metrics
    mae = mean_absolute_error(test_data_original, forecast_original)
    mse = mean_squared_error(test_data_original, forecast_original)
    rmse = np.sqrt(mse)
    mape = (abs((test_data_original - forecast_original) / test_data_original).mean()) * 100

    # Save metrics to the summary file
    with open(f"county_timeseries_models/{county}_model_summary.txt", "a") as f:
        f.write(f"\n\nEvaluation Metrics:\n")
        f.write(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")
        f.write(f"Mean Absolute Error (MAE): {mae:.2f}\n")
        f.write(f"Mean Squared Error (MSE): {mse:.2f}\n")
        f.write(f"Root Mean Squared Error (RMSE): {rmse:.2f}\n")

    print(f"Model for {county} completed and saved.")


Model for Baringo completed and saved.
Model for Bomet completed and saved.
Model for Bungoma completed and saved.
Model for Busia completed and saved.
Model for Elgeyo-Marakwet completed and saved.
Model for Embu completed and saved.
Model for Garissa completed and saved.
Model for Homa-bay completed and saved.
Model for Isiolo completed and saved.
Model for Kajiado completed and saved.
Model for Kakamega completed and saved.
Model for Kericho completed and saved.
Model for Kiambu completed and saved.
Model for Kilifi completed and saved.
Model for Kirinyaga completed and saved.
Model for Kisii completed and saved.
Model for Kisumu completed and saved.
Model for Kitui completed and saved.
Model for Kwale completed and saved.
Model for Laikipia completed and saved.
Model for Lamu completed and saved.
Model for Machakos completed and saved.
Model for Makueni completed and saved.
Model for Mandera completed and saved.
Model for Meru completed and saved.
Model for Migori completed and sav